In [ ]:
# Use differential evolution to maximize the compression strength
def maximize_obj_func_lgbm(x):
    return -model_lgbm.predict([x])

def maximize_obj_func_lr(x):
    return -model_lr.predict([x])


def run_optimizer(x,y,model):
    bounds = [(df[x].min(), df[x].max()), (df[y].min(), df[y].max())]
    iterations_data = []
    def callback_lgbm(xk, convergence):
        # Store information at each iteration
        iterations_data.append({
            'iteration': len(iterations_data) + 1,        # Store the iteration number
            'solution': np.copy(xk),                      # Store a copy of the solution
            'objective_value': maximize_obj_func_lgbm(xk),    # Store the objective function value at the current solution
            'convergence': convergence                    # Store the convergence value
        })
    def callback_lr(xk, convergence):
        # Store information at each iteration
        iterations_data.append({
            'iteration': len(iterations_data) + 1,        # Store the iteration number
            'solution': np.copy(xk),                      # Store a copy of the solution
            'objective_value': maximize_obj_func_lr(xk),    # Store the objective function value at the current solution
            'convergence': convergence                    # Store the convergence value
        })
        
    if model == "model_lgbm":
        result = differential_evolution(maximize_obj_func_lgbm, bounds,callback=callback_lgbm)
    elif model == "model_lr":
        result = differential_evolution(maximize_obj_func_lr, bounds,callback=callback_lr)

    optimized_x = result.x[0]
    optimized_y = result.x[1]
    optimized_obj = -result.fun

    print(f"Optimized X: {optimized_x}")
    print(f"Optimized Y: {optimized_y}")
    print(f"Optimized Strength: {optimized_obj}")

    return optimized_x,optimized_y,optimized_obj,iterations_data

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_optimized_obj_func(x_values, y_values, model):

    # Run the optimizer
    output = run_optimizer(x_values, y_values, model)

    # Create a grid of x and y values for the surface plot
    x_grid = df[x_values]
    y_grid = df[y_values]
    x_grid, y_grid = np.meshgrid(x_grid, y_grid)
    # Use the trained model to predict z values for the grid
    if model == "model_lgbm":
        z_grid = model_lgbm.predict(
            np.column_stack((x_grid.ravel(), y_grid.ravel()))
        ).reshape(x_grid.shape)
    elif model == "model_lr":
        z_grid = model_lr.predict(
            np.column_stack((x_grid.ravel(), y_grid.ravel()))
        ).reshape(x_grid.shape)

    plt.figure(figsize=(5, 3))
    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "surface"}]])

    # Add the surface plot
    surface = go.Surface(
        x=x_grid, y=y_grid, z=z_grid, colorscale="Viridis", opacity=0.7
    )
    fig.add_trace(surface)

    # Add the scatter plot for the optimized point
    scatter = go.Scatter3d(
        x=[output[0]],
        y=[output[1]],
        z=[output[2]],
        mode="markers",
        marker=dict(size=5, color="red"),
        name="Optimized Point",
    )
    fig.add_trace(scatter)

    # Set labels and title
    fig.update_layout(
        scene=dict(
            xaxis_title=f"{x_values}",
            yaxis_title=f"{y_values}",
            zaxis_title="Congrete Compressive Strength",
        ),
        width=700,
        height=700,
        title=f"3D Plot of Model Predictions using {model}",
    )

    # Show the plot
    fig.show()

    print(f"Optimized X: {output[0]}")
    print(f"Optimized Y: {output[1]}")
    print(f"Optimized Obj: {output[2]}")

    iteration = output[3]
    # Extract the convergence values
    convergence = [data["convergence"] for data in iteration]

    # Extract the solution values
    solution = [data["solution"] for data in iteration]

    # Extract the objective function values
    objective_values = [data["objective_value"] for data in iteration]

    # Plot the solution values and convergence values
    plt.figure(figsize=(5, 3))
    plt.plot(convergence)
    plt.title("Convergence Plot")
    plt.xlabel("Iteration")
    plt.ylabel("Convergence Value")
    plt.grid(True)
    plt.show()

    # Plot the solution values and convergence values
    plt.figure(figsize=(5, 3))
    plt.plot(solution)
    plt.title("Convergence of Decision Variables")
    plt.xlabel("Iteration")
    plt.ylabel("Objective Function Value")
    plt.grid(True)
    plt.show()

    # Plot the solution values and convergence values
    plt.figure(figsize=(5, 3))
    plt.plot(objective_values)
    plt.title("Convergence of Objective Function Values")
    plt.xlabel("Iteration")
    plt.ylabel("Objective Function Value")
    plt.grid(True)
    plt.show()